<a href="https://colab.research.google.com/github/theiaa7/PythonLC/blob/main/P4_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scraping
Menggunakan Beautiful Soup

In [ ]:
import requests #Modul request ini memperbolehkan kita untuk meminta HTTP Request melalui Python
import pandas as pd # Python Data Analysis library spt biasa
import seaborn as sns # ini sama kayak matplotlib, untuk visualisasi statistik data
import matplotlib.pyplot as plt # ^
from bs4 import BeautifulSoup as bs4 # Library untuk pengambil(PARSING) data dari HTML dan XML

Text scraping ini mengambil data melalui request http. dalam kasus kali ini kita menggunakan wikipedia, tapi perlu di ingat tidak semua website dapat memberikan data dengan metode ini, hanya website public yang tidak membutuhkan login ataupun keamanan khusus saja.

untuk yang lain, kadang membutuhkan token login maupun API dari website/apl tertentu.

In [ ]:
benua_req = requests.get(
    "https://simple.wikipedia.org/wiki/List_of_countries_by_continents").text # nah ini untuk mengambil semua feature yang ada di dalam website
    # dengan url spesifik kita akan mendapatkan raw html file dari http request tadi dalam bentuk teks
benua_req

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>List of countries by continents - Simple English Wikipedia, the free encyclopedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-cl

In [ ]:
benua_soup = bs4(benua_req,"lxml")  # bs4 itu lib nya, benua itu textnya yang berupa html/xml,
                                    # lxml itu adalah toolkit nya untuk prosessing file text html/xml
benua = benua_soup.find_all('h2' > 'span', {"class":"mw-headline"})
# Line 3 ini, pake fitur find_all dengan mencari tag <h2> dan didalam nya ada span dengan class:mw-headline
benua # setelah semua di temukan, maka akan jadi seperti ini
# notes untuk beberapa website tidak terlalu sama, jadi user harus mempelajari struktur html terlebih dahulu

[<span class="mw-headline" id="Africa">Africa</span>,
 <span class="mw-headline" id="Asia">Asia</span>,
 <span class="mw-headline" id="Europe">Europe</span>,
 <span class="mw-headline" id="North_America">North America</span>,
 <span class="mw-headline" id="South_America">South America</span>,
 <span class="mw-headline" id="Oceania">Oceania</span>,
 <span class="mw-headline" id="Notes">Notes</span>,
 <span class="mw-headline" id="References">References</span>,
 <span class="mw-headline" id="Other_website">Other website</span>]

In [ ]:
unwanted_words = ["Antarctica","References","Other websites"] #buang beberapa ini, karena ga relevan
target_benua = [continent.text for continent in benua if continent.text not in unwanted_words]
target_benua

['Africa',
 'Asia',
 'Europe',
 'North America',
 'South America',
 'Oceania',
 'Notes',
 'Other website']

In [ ]:
ol_html = benua_soup.find_all('ol') # temukan semua ordered list di benua_soup
negara = [countries.find_all('li',{"class": None, "id": None}) for countries in ol_html]
# bikin variable baru, terus find semua tag <li> dengan class dan id none di ol_html
negara

[[], [], [], []]

In [ ]:
negara_benua = [] # variable negara di setiap benua
for items in negara:
    a_negara = [] # variable negara
    if items:
        for country in items:
            a_negara = [country.find('a').text for country in items if country.find('a')] # ngesplit array negara, untuk masing" benua
        negara_benua.append(a_negara) #append ini buat ngelist single item, ke array yang ada di countries ke negara benua
negara_benua

[]

In [ ]:
countries_continent_category_df = pd.DataFrame(
    zip(negara_benua, target_benua), columns=['Country', 'Continent'])
countries_continent_category_df # mengubah list array menjadi tabel

,Country,Continent


In [ ]:
countries_continent_category_df = countries_continent_category_df.explode(
    'Country').reset_index(drop=True)
countries_continent_category_df # mengubah index tabel menjadi per-negara daripada benua

,Country,Continent


In [ ]:
countries_continent_category_df = countries_continent_category_df.replace('United States of America', 'United States')
countries_continent_category_df

,Country,Continent


In [ ]:
countries_score_page = requests.get("https://www.worldometers.info/world-population/population-by-country/")
countries_score_soup = bs4(countries_score_page.content,'lxml')
countries_score_soup

<!DOCTYPE html>
<!--[if IE 8]> <html lang="en" class="ie8"> <![endif]--><!--[if IE 9]> <html lang="en" class="ie9"> <![endif]--><!--[if !IE]><!--><html lang="en"> <!--<![endif]--> <head> <meta charset="utf-8"/> <meta content="IE=edge" http-equiv="X-UA-Compatible"/> <meta content="width=device-width, initial-scale=1" name="viewport"/> <title>Population by Country (2024) - Worldometer</title><meta content="List of countries and dependencies in the world ranked by population, from the most populated. Growth rate, median age, fertility rate, area, density, population density, urbanization, urban population, share of world population." name="description"/><!-- Favicon --><link href="/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="/favicon/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/><link href="/favicon/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/><link href="/favicon/apple-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/><link h

In [ ]:
countries_score_table = countries_score_soup.find('table', {'class':'table'})
countries_score_table

<table cellspacing="0" class="table table-striped table-bordered" id="example2" width="100%"> <thead> <tr> <th>#</th> <th>Country (or dependency)</th> <th>Population<br/> (2023)</th> <th>Yearly<br/> Change</th> <th>Net<br/> Change</th> <th>Density<br/> (P/Km²)</th> <th>Land Area<br/> (Km²)</th> <th>Migrants<br/> (net)</th> <th>Fert.<br/> Rate</th> <th>Med.<br/> Age</th> <th>Urban<br/> Pop %</th> <th>World<br/> Share</th> </tr> </thead> <tbody> <tr> <td>1</td> <td style="font-weight: bold; font-size:15px; text-align:left"><a href="/world-population/india-population/">India</a></td> <td style="font-weight: bold;">1,428,627,663</td> <td>0.81 %</td> <td>11,454,490</td> <td>481</td> <td>2,973,190</td> <td>-486,136</td> <td>2.0</td> <td>28</td> <td>36 %</td> <td>17.76 %</td> </tr> <tr> <td>2</td> <td style="font-weight: bold; font-size:15px; text-align:left"><a href="/world-population/china-population/">China</a></td> <td style="font-weight: bold;">1,425,671,352</td> <td>-0.02 %</td> <td>-21

In [ ]:
countries_score_df = pd.read_html(str(countries_score_table))
countries_score_df

[       # Country (or dependency)  Population  (2023) Yearly  Change  \
 0      1                   India          1428627663         0.81 %   
 1      2                   China          1425671352        -0.02 %   
 2      3           United States           339996563         0.50 %   
 3      4               Indonesia           277534122         0.74 %   
 4      5                Pakistan           240485658         1.98 %   
 ..   ...                     ...                 ...            ...   
 229  230              Montserrat                4386        -0.09 %   
 230  231        Falkland Islands                3791         0.29 %   
 231  232                    Niue                1935         0.05 %   
 232  233                 Tokelau                1893         1.18 %   
 233  234                Holy See                 518         1.57 %   
 
      Net  Change  Density  (P/Km²)  Land Area  (Km²)  Migrants  (net)  \
 0       11454490               481           2973190       

In [ ]:
countries_score_df = countries_score_df[0]
countries_score_df

,#,Country (or dependency),Population (2023),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,1,India,1428627663,0.81 %,11454490,481,2973190,-486136,2.0,28.0,36 %,17.76 %
1,2,China,1425671352,-0.02 %,-215985,152,9388211,-310220,1.2,39.0,65 %,17.72 %
2,3,United States,339996563,0.50 %,1706706,37,9147420,999700,1.7,38.0,83 %,4.23 %
3,4,Indonesia,277534122,0.74 %,2032783,153,1811570,-49997,2.1,30.0,59 %,3.45 %
4,5,Pakistan,240485658,1.98 %,4660796,312,770880,-165988,3.3,21.0,35 %,2.99 %
...,...,...,...,...,...,...,...,...,...,...,...,...
229,230,Montserrat,4386,-0.09 %,-4,44,100,0,1.6,44.0,11 %,0.00 %
230,231,Falkland Islands,3791,0.29 %,11,0,12170,0,1.6,40.0,62 %,0.00 %
231,232,Niue,1935,0.05 %,1,7,260,0,2.4,36.0,41 %,0.00 %
232,233,Tokelau,1893,1.18 %,22,189,10,0,2.6,27.0,0 %,0.00 %


In [ ]:
countries_score_df = countries_score_df.rename(columns={"Country (or dependency)":"Country","Population (2020)":"Population"})
countries_score_df

,#,Country,Population (2023),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,1,India,1428627663,0.81 %,11454490,481,2973190,-486136,2.0,28.0,36 %,17.76 %
1,2,China,1425671352,-0.02 %,-215985,152,9388211,-310220,1.2,39.0,65 %,17.72 %
2,3,United States,339996563,0.50 %,1706706,37,9147420,999700,1.7,38.0,83 %,4.23 %
3,4,Indonesia,277534122,0.74 %,2032783,153,1811570,-49997,2.1,30.0,59 %,3.45 %
4,5,Pakistan,240485658,1.98 %,4660796,312,770880,-165988,3.3,21.0,35 %,2.99 %
...,...,...,...,...,...,...,...,...,...,...,...,...
229,230,Montserrat,4386,-0.09 %,-4,44,100,0,1.6,44.0,11 %,0.00 %
230,231,Falkland Islands,3791,0.29 %,11,0,12170,0,1.6,40.0,62 %,0.00 %
231,232,Niue,1935,0.05 %,1,7,260,0,2.4,36.0,41 %,0.00 %
232,233,Tokelau,1893,1.18 %,22,189,10,0,2.6,27.0,0 %,0.00 %


In [ ]:
countries_score_df = countries_score_df.replace('N.A.', '0')
countries_score_df

,#,Country,Population (2023),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,1,India,1428627663,0.81 %,11454490,481,2973190,-486136,2.0,28.0,36 %,17.76 %
1,2,China,1425671352,-0.02 %,-215985,152,9388211,-310220,1.2,39.0,65 %,17.72 %
2,3,United States,339996563,0.50 %,1706706,37,9147420,999700,1.7,38.0,83 %,4.23 %
3,4,Indonesia,277534122,0.74 %,2032783,153,1811570,-49997,2.1,30.0,59 %,3.45 %
4,5,Pakistan,240485658,1.98 %,4660796,312,770880,-165988,3.3,21.0,35 %,2.99 %
...,...,...,...,...,...,...,...,...,...,...,...,...
229,230,Montserrat,4386,-0.09 %,-4,44,100,0,1.6,44.0,11 %,0.00 %
230,231,Falkland Islands,3791,0.29 %,11,0,12170,0,1.6,40.0,62 %,0.00 %
231,232,Niue,1935,0.05 %,1,7,260,0,2.4,36.0,41 %,0.00 %
232,233,Tokelau,1893,1.18 %,22,189,10,0,2.6,27.0,0 %,0.00 %


In [ ]:
merged_df = pd.merge(countries_score_df, countries_continent_category_df, how='inner', on='Country')
merged_df.to_csv('final_result.csv')
merged_df

,#,Country,Population (2023),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share,Continent


## Analysis
Kenapa kita gak ambil langsung aja data ranking nya?
itu bisa sekali, tapi, disini kita akan membuat kenapa kita menambahkan field continent(benua) ke dataframe kita.

field benua akan membantu kita untuk analisis data, dan klasifikasi berdasarkan benua.


In [ ]:
final_result_df_score_index = pd.read_csv('final_result.csv', index_col=2)
ax = final_result_df_score_index['Population'].plot(kind='bar', figsize=(30,5), title="Ini data populasi")
ax.set_ylabel("Populasi")

KeyError: 'Population'

In [ ]:
sns.lmplot(x='Fert. Rate', y='Population', data=merged_df, fit_reg=False)

In [ ]:
sns.lmplot(x='Fert. Rate', y='Population', data=merged_df, fit_reg=False, hue='Continent')